In [26]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

In [27]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [28]:
epochs.apply_baseline((None, 0))

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [29]:
# epochs.plot()

### Reject Artifacts based on channel signal amplitude

In [30]:
reject_criteria = dict(mag = 3000 * (10 ** - 15),
                      grad = 3000 * (10 ** - 13),
                      eeg = 150 * (10 ** - 6),
                      eog = 200 * (10 ** - 6))

flat_criteria = dict(mag = 1 * (10 ** - 15),
                    grad = 1 * (10 ** - 13),
                    eeg = 1 * (10 ** - 6))

In [31]:
epochs.drop_bad(reject = reject_criteria, flat = flat_criteria)

Number of events,271
Events,Auditory/Left: 57Auditory/Right: 61Button: 15Smiley: 14Visual/Left: 67Visual/Right: 57
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [32]:
epochs.plot_drop_log()

<Figure size 640x476 with 1 Axes>

In [33]:
epochs['Visual'].plot_image()

[<Figure size 640x476 with 3 Axes>,
 <Figure size 640x476 with 3 Axes>,
 <Figure size 640x476 with 3 Axes>]

In [34]:
epochs.plot_sensors(ch_type = 'eeg')

<Figure size 640x640 with 1 Axes>

In [37]:
epochs['Visual'].plot_image(picks = 'EEG 060')

[<Figure size 640x476 with 4 Axes>]

### SSP

In [38]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject = '01',
                             session = '01',
                             task = 'audiovisual',
                             run = '01',
                             datatype = 'meg',
                             root = bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq = 0.1, h_freq = 40)

ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad = 1, n_mag = 1, n_eeg = 0,
                                                           average = True)

eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad = 1, n_mag = 1, n_eeg = 0,
                                                           average = True)

C:\Users\hp\AppData\Roaming\Python\Python37\site-packages\mne_bids\read.py:406: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw.set_channel_types(channel_type_dict)


In [40]:
ecg_projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | ECG-planar--0.200-0.400-PCA-01, active : False, n_channels : 203>,
 <Projection | ECG-axial--0.200-0.400-PCA-01, active : False, n_channels : 102>]

In [41]:
eog_projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>]

In [39]:
projs = ecg_projs + eog_projs
projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | ECG-planar--0.200-0.400-PCA-01, active : False, n_channels : 203>,
 <Projection | ECG-axial--0.200-0.400-PCA-01, active : False, n_channels : 102>,
 <Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>]

In [44]:
epochs.add_proj(projs)
epochs.plot()

<MNEBrowseFigure size 1920x953 with 5 Axes>

In [46]:
epochs_cleaned = epochs.copy().apply_proj()

epochs_cleaned['Visual'].plot_image()
epochs_cleaned['Visual'].plot_image(picks = 'EEG 060')

[<Figure size 640x476 with 4 Axes>]